In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// set parameterz
int[] core_sweep = {4,64};
int[] PolyDegS = new int[] {2};
int[] ResArray = new int[] {20};
int MemoryPerCore = 2000;

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,,,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,"[ { \\hpccluster\hpccluster-scratch\weber == : LocalMountPath: \\hpccluster\hpccluster-scratch\weber, PathAtRemote: } ]",FDY\weber,<null>,DC2,[ HPCCLUSTER2 ],Normal,True,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,<null>,dotnet,,,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,HHLR,dotnet,,"[ { W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa: LocalMountPath: W:\work\scratch\jw52xeqa, PathAtRemote: /work/scratch/jw52xeqa } ]",jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512 ]",/home/jw52xeqa/Deployerie,special00006,<null>,False


In [ ]:
using BoSSS.Application.BoSSSpad;

In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
var myBatch = (MsHPC2012Client)ExecutionQueues[1];

In [ ]:

string WFlowName = "test";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'test'.


In [ ]:
string dirname = "DB_rotSphereBenchmark";
string winpath = @"H:\weber\"+dirname;
//string winpath = @"\\\\hpccluster\\hpccluster-scratch\\weber\"+dirname;
var myDB = OpenOrCreateDatabase(winpath);
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDB.Path,""));

Opening existing database 'H:\weber\DB_rotSphereBenchmark'.


In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
}

In [ ]:
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;

In [ ]:
myDB.Grids

#0: { Guid = 55fdd8ad-65ea-4f41-ae70-9c0d22396da8; Name = RotBenchmarkGrid; Cell Count = 8000; Dim = 3 }


In [ ]:
double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;
int SpaceDim=3;

In [ ]:
List<string> controls = new List<string>();

In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();
foreach(var Res in ResArray){
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    var tmp = myDB.Grids.Where(g=>g.Name.Equals(gname)&&g.NumberOfCells==Res*Res*Res);
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;
    }
    switch (SpaceDim) {
        case 2:
        grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);
        break;

        case 3:
        grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
        break;

        default:
        throw new ArgumentOutOfRangeException();
    }
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("debug", MakeDebugPart);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        if (SpaceDim == 3)
            z = X[2];

        return 2;
    });
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
} Grids

Grid found: RotBenchmarkGrid


#0: { Guid = 55fdd8ad-65ea-4f41-ae70-9c0d22396da8; Name = RotBenchmarkGrid; Cell Count = 8000; Dim = 3 }


In [ ]:
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

In [ ]:
bool useAMR = false;
bool useLoadBal = true;
int NoOfTimeSteps = 1;
bool Steady = true;

In [ ]:
Func<IGridInfo, int, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================

    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}", J, k);
    C.Tags.Add(SpaceDim + "D");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 1;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    //C.SetDefaultEstimatorFactory();
    C.DynamicLoadBalancing_ImbalanceThreshold = -0.1;


    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double muA = 1;
    const double anglev = 10;
    const double partRad = 0.261;
    double[] pos = new double[SpaceDim];

    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Shape.Sphere);
    //C.SetDefaultVelocityAtIB(pos,anglev,SpaceDim);
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    //C.DefineSphere(pos,partRad,SpaceDim);
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Wall");

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.LinearSolver.NoOfMultigridLevels = 5;
    C.LinearSolver.ConvergenceCriterion = 1E-8;
    C.LinearSolver.MaxSolverIterations = 100;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 10000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-6;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(2);
        C.AMR_startUpSweeps = 1;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = 0.1;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
    C.dtFixed = dt;
    return C;
};

In [ ]:
List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids){
    foreach(int k in PolyDegS){
        controls.Add(GenXNSECtrl(grd,k));
    }
} controls.Select(s=>s.SessionName)

index,value
0,J8000_k2


In [ ]:
// ==========
// Launch Job
// ==========
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;

In [ ]:
controls.Select(s=>s.SessionName)

index,value
0,J8000_k2


In [ ]:
foreach(int cores in core_sweep){
    foreach(var ctrl in controls){
        var rnd = new Random();
        ctrl.SessionName += "_"+rnd.Next(); 
        var aJob   = new Job("rotting_sphere"+ctrl.SessionName,typeof(XNSE));
        aJob.SetControlObject(ctrl);
        //aJob.SetControlStatement(String.Format(@"cs:BoSSS.Application.XNSE_Solver.HardcodedControl.Rotating_Cube({0}, {1}, {2},true,{3},true,false,true,false)",2,10,3,10));
        aJob.NumberOfMPIProcs         = cores;
        aJob.ExecutionTime            = "1:00:00";
        aJob.UseComputeNodesExclusive = true;
        aJob.Activate(myBatch);
    }
}

Deploying job rotting_sphereJ8000_k2_1100533799 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\weber\dply\test-XNSE_Solver2021Sep01_154455
copied 49 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.

